In [35]:
# Import the libraries to process data:
import yfinance as yf
import pandas as pd
import numpy as np
# Import the libraries to process dates:
from datetime import datetime, timedelta, date

In [36]:
### Question 1: Fetch data for Google over the past year from 2024-01-01 till now:
def fetch_raw_data(ticker="GOOG") -> pd.DataFrame:
    # 1) Define the time window from where to gather data: = [startDate:endDate]:
    endDate = date.today() # Format: 2024-12-12
    startDate = endDate.replace(month=1, day=1) # Format: 2024-01-01
    # 2) Attempt to fetch Financial data for GOOG for the defined time frame:
    try:
        df = yf.download(tickers=ticker, start=startDate, end=endDate, progress=False)
    except Exception as e:
        print(f"Couldn't dowload data for the stock {ticker}: {str(e)}")
        return pd.DataFrame()

    # 3) Assess if there is a multi index in the DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    # 4) Reset the index:
    df.reset_index(inplace=True)
    # 5) Only keep the necessary features:
    df = df[["Date", "Open", "High", "Low", "Close", "Adj Close"]]
    # 6) Add the ticker symbol as feature:
    df["Ticker"] = ticker
    # 7) Return the dataframe:
    return df

raw_df = fetch_raw_data()
print(raw_df)
    

    

Price       Date        Open        High         Low       Close   Adj Close  \
0     2024-01-02  139.600006  140.615005  137.740005  139.559998  139.060349   
1     2024-01-03  138.600006  141.089996  138.429993  140.360001  139.857483   
2     2024-01-04  139.850006  140.634995  138.009995  138.039993  137.545776   
3     2024-01-05  138.352005  138.809998  136.850006  137.389999  136.898117   
4     2024-01-08  138.000000  140.639999  137.880005  140.529999  140.026871   
..           ...         ...         ...         ...         ...         ...   
236   2024-12-09  175.714996  178.039993  175.399994  177.100006  177.100006   
237   2024-12-10  184.535004  188.029999  182.669998  186.529999  186.529999   
238   2024-12-11  186.699997  196.889999  186.259995  196.710007  196.710007   
239   2024-12-12  196.300003  196.705002  193.279999  193.630005  193.630005   
240   2024-12-13  192.710007  194.339996  191.259995  191.380005  191.380005   

Price Ticker  
0       GOOG  
1       G

In [37]:
### Question 2: Transform the dataframe to a file like string output:
from io import StringIO
import heapq

def convert_df_to_file_output(df: pd.DataFrame) -> str:
    # 1) Instantiate a min heap to process data in Ascending order (chronological order):
    min_heap = []
    # 2) Iterrate through each lines yielded by the iterrows() generator:
    for _, row in df.iterrows():
        # 2.1) Push the data into the min_heap:
        heapq.heappush(min_heap, (row["Ticker"], row["Date"], row["Open"], row["High"], row["Low"], row["Close"], row["Adj Close"]))
    index = 1
    # 3) Instantiate a StringIO object to redirect standard output to the object like in a file:
    result = StringIO()

    # 4) Iterrate through the min heap pop each lines in chronological orders and write S/O to the StringIO object:
    while min_heap:
        # 4.1) Get each features from each line:
        ticker, date, open_price, high, low, close, adj_close = heapq.heappop(min_heap)
        # 4.2) Redirect S/O to the StringIO object:
        print(f"{index} {ticker}, {date}, {open_price}, {high}, {low}, {close}, {adj_close}",file=result)
        # 4.3) Increment the index:
        index += 1

    # 5) Reset the pointer to point at the beginning of the StringIO object:
    result.seek(0)
    # 6) Return the content of the file:
    return result.getvalue()

file_data = convert_df_to_file_output(raw_df)
print(file_data)

1 GOOG, 2024-01-02 00:00:00, 139.60000610351562, 140.61500549316406, 137.74000549316406, 139.55999755859375, 139.0603485107422
2 GOOG, 2024-01-03 00:00:00, 138.60000610351562, 141.08999633789062, 138.42999267578125, 140.36000061035156, 139.85748291015625
3 GOOG, 2024-01-04 00:00:00, 139.85000610351562, 140.63499450683594, 138.00999450683594, 138.0399932861328, 137.5457763671875
4 GOOG, 2024-01-05 00:00:00, 138.3520050048828, 138.80999755859375, 136.85000610351562, 137.38999938964844, 136.8981170654297
5 GOOG, 2024-01-08 00:00:00, 138.0, 140.63999938964844, 137.8800048828125, 140.52999877929688, 140.02687072753906
6 GOOG, 2024-01-09 00:00:00, 140.05999755859375, 142.8000030517578, 139.7899932861328, 142.55999755859375, 142.0496063232422
7 GOOG, 2024-01-10 00:00:00, 142.52000427246094, 144.52499389648438, 142.4600067138672, 143.8000030517578, 143.28517150878906
8 GOOG, 2024-01-11 00:00:00, 144.89500427246094, 146.66000366210938, 142.21499633789062, 143.6699981689453, 143.15562438964844
9

In [38]:
### Problem 3: Convert the file like string input back into a dataframe:
import re

def convert_file_str_to_df(str_input: str) -> pd.DataFrame:
    # 1) remove any leading and trailing whitespace and split the string into lines:
    lines = str_input.strip().split('\n')
    # 2) Instantiate a list that will store lists: => List of list:
    data = []

    # 3) Iterrate over each lines:
    for line in lines:
        # 3.1) Split each component of a line each time this pattern is met: 0+sp , 0+wp:
        parts = re.split(r'\s*,\s*', line)
        
        # 3.2) Ensure there are 7 features:
        if len(parts)!=7:
            print("Error There isn't 7 features")
            return pd.DataFrame
            
        # 3.3) Extract each features:
        ticker = parts[0].split()[1]
        date = parts[1].strip()
        open_price = float(parts[2].strip())
        high = float(parts[3].strip())
        low = float(parts[4].strip())
        close = float(parts[5].strip())
        adj_price = float(parts[6].strip())
        # 3.4) Combine the data into a list and append it to the list:
        data.append([ticker, date, open_price, high, low, close, adj_price])

    # 4) Create a dataframe and add all the columns:
    df = pd.DataFrame(data, columns=["Ticker", "Date", "Open", "High", "Low", "Close", "Adj_price"])
    # 4.1) Convert the data into a datatime object:
    df["Date"] = pd.to_datetime(df["Date"])
    # 4.2) Round numerical values to 3 decimales:
    df[["Open", "High", "Low", "Close", "Adj_price"]] = df[["Open", "High", "Low", "Close", "Adj_price"]].round(3)
    # 4.3) Return the dataframe:
    return df

test = convert_file_str_to_df(file_data)
print(test)

    

    Ticker       Date     Open     High     Low   Close  Adj_price
0     GOOG 2024-01-02  139.600  140.615  137.74  139.56    139.060
1     GOOG 2024-01-03  138.600  141.090  138.43  140.36    139.857
2     GOOG 2024-01-04  139.850  140.635  138.01  138.04    137.546
3     GOOG 2024-01-05  138.352  138.810  136.85  137.39    136.898
4     GOOG 2024-01-08  138.000  140.640  137.88  140.53    140.027
..     ...        ...      ...      ...     ...     ...        ...
236   GOOG 2024-12-09  175.715  178.040  175.40  177.10    177.100
237   GOOG 2024-12-10  184.535  188.030  182.67  186.53    186.530
238   GOOG 2024-12-11  186.700  196.890  186.26  196.71    196.710
239   GOOG 2024-12-12  196.300  196.705  193.28  193.63    193.630
240   GOOG 2024-12-13  192.710  194.340  191.26  191.38    191.380

[241 rows x 7 columns]


In [40]:
### Question 4: Remove outliers with the IQR method:
### IQR = Q3 - Q1 => Where 50% of the data is located:
### Aim of IQR: Remove outliers outside of the bound: I1 = Q1 - 1.5*IQR  and I3 = Q3 + 1.5* IQR
def remove_outliers_with_iqr(df: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    # 1) Create a copy of the DataFrame to avoid modifying the original:
    filtered_df = df.copy()
    # 2) Identify numeric columns in the DataFrame:
    numeric_columns = filtered_df.select_dtypes(include=["number"]).columns
    # 3) Store row indices of outliers to avoid duplicates:
    outlier_indices = set()

    # 4) Iterate over each features in the DataSet:
    for column in numeric_columns:
        # 4.1) Compute the 25th & 75th percentile:
        Q1 = filtered_df[column].quantile(0.25)
        Q3 = filtered_df[column].quantile(0.75)
        # 4.2) Compute the IQR = Q3 - Q1:
        IQR = Q3 - Q1
        # 4.3) Compute the lower & upper bound:
        lower_bound = Q1 - 1.5*IQR
        upper_bound = Q3 + 1.5*IQR
        # 4.4) Identify outliers for the current feature:
        column_outlier = filtered_df[(filtered_df[column]<lower_bound) | (filtered_df[column]>upper_bound)]
        # 4.5) Add the idices of the outlier to the set (unique indices only):
        outlier_indices.update(column_outlier.index)
        # 4.6) Remove outliers from the filtered DataFrame:
        filtered_df = filtered_df[(filtered_df[column]>=lower_bound) & (filtered_df[column]<=upper_bound)]

    # 5) Create a DataFrame for the outliers based on the stored indices:
    outliers_df = df.loc[list(outlier_indices)]
    # 6) Reset the indices of both DataFrame:
    filtered_df.reset_index(drop=True, inplace=True)
    outliers_df.reset_index(drop=True, inplace=True)
    # 7) Remove both DataFrame:
    return filtered_df, outliers_df

filtered_df, outliers_df = remove_outliers_with_iqr(test)
print("Filtered DataFrame (Outliers Removed):")
print(filtered_df)
print("\nOutliers DataFrame:")
print(outliers_df)

Filtered DataFrame (Outliers Removed):
    Ticker       Date     Open     High     Low   Close  Adj_price
0     GOOG 2024-01-02  139.600  140.615  137.74  139.56    139.060
1     GOOG 2024-01-03  138.600  141.090  138.43  140.36    139.857
2     GOOG 2024-01-04  139.850  140.635  138.01  138.04    137.546
3     GOOG 2024-01-05  138.352  138.810  136.85  137.39    136.898
4     GOOG 2024-01-08  138.000  140.640  137.88  140.53    140.027
..     ...        ...      ...      ...     ...     ...        ...
236   GOOG 2024-12-09  175.715  178.040  175.40  177.10    177.100
237   GOOG 2024-12-10  184.535  188.030  182.67  186.53    186.530
238   GOOG 2024-12-11  186.700  196.890  186.26  196.71    196.710
239   GOOG 2024-12-12  196.300  196.705  193.28  193.63    193.630
240   GOOG 2024-12-13  192.710  194.340  191.26  191.38    191.380

[241 rows x 7 columns]

Outliers DataFrame:
Empty DataFrame
Columns: [Ticker, Date, Open, High, Low, Close, Adj_price]
Index: []
